# Este notebook lo usaré para hacer el modelado del TFM

Referencias:
* https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c
* https://medium.com/@juan.7655/cleansing-messy-excel-files-with-pandas-95a2ba3d8b2d
* https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/
* https://datatofish.com/convert-pandas-dataframe-to-list/
* https://www.geeksforgeeks.org/python-pandas-dataframe-insert/
* https://thispointer.com/python-pandas-drop-columns-in-dataframe-by-label-names-or-by-index-positions/

In [1]:
import pandas as pd
import math
import timeit

path = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\" #Path para mi laptop en casa
#path = "F:\\DATA_TFM\\CLEANDATA\\" #Path para mi PC en el trabajo

archivo = "df_best_hits_limpio.csv"
file_hits = path+archivo

archivo = "data_aafit_bbfit_muon.csv"
file_aafit = path+archivo

## Analizando los datos (dataset).

<font size=3>Después de todo el trabajo de preprocesamiento y limpieza, tenemos que, estos están por el momento divididos en dos partes.

* Dataframe 1, el cual contiene los hits, mismo que proviene del archivo llamado df_best_hits_limpio_v2.csv
* Dataframe 2, el cual contiene la data de aafit, bbfit y el muon por frameID (evento), mismo que proviene del archivo data_aafit_bbfit_muon.csv
</font>

In [2]:
#Veamos como es el archivo del dataset relativo a los hits
df_hits = pd.read_csv(file_hits)

#En cargar tarda "bastante" alrededor de 5 min en la PC del trabajo (Core i5, 32GB RAM, Disco SSD.), 
#en la laptop de casa, casi 8 min.

In [5]:
#Obtenemos información del dataframe
df_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63520301 entries, 0 to 63520300
Data columns (total 17 columns):
Unnamed: 0         int64
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
#hit               int64
lin_det            int64
piso_det           int64
mód_opt            int64
X'                 float64
Y'                 float64
Z'                 float64
azimut             float64
zenit              float64
tiempo             float64
amp                float64
frec_hits          float64
dtypes: float64(8), int64(9)
memory usage: 8.0 GB


In [6]:
#Mostramos las 10 primeras líneas para "ver" como son
df_hits.head(10)

,Unnamed: 0,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,amp,frec_hits
0,0,25800,73557,0,2,76,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,296.559100,1.089169,327.587128
1,1,25800,73557,0,2,77,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,137.146191,0.879543,327.587128
2,2,25800,73557,0,2,78,4,19,1,44.584223,9.539780,90.040108,2.292014,2.356193,153.189412,0.772948,281.085968
3,3,25800,73557,0,2,67,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,168.470446,1.070132,82.550049
4,4,25800,73557,0,2,68,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,897.750991,0.985427,82.550049
5,5,25800,73557,0,2,69,4,20,1,44.485280,9.426792,104.567108,2.552615,2.356193,167.488297,4.492627,73.738098
6,6,25800,73557,0,2,70,4,20,2,44.928075,8.529054,104.567108,-1.636175,2.356193,171.860943,1.107187,89.874268
7,7,25800,73557,0,2,71,4,21,0,44.788333,9.655823,119.094108,1.882990,2.356193,201.236802,16.856011,60.100555
8,8,25800,73557,0,2,72,4,21,1,44.578323,8.677005,119.094108,-2.305800,2.356193,204.882482,3.327063,52.337646
9,9,25800,73557,0,2,73,4,21,2,45.530937,8.984459,119.094108,-0.211405,2.356193,1035.010812,0.580996,61.416626


In [7]:
#Es extraño pero tenemos una columnna unnamed la cual eliminaré, insisto es extraño,
#puesto que en la extracción no se mostró, y cuando grabé el df lo hice con el parámetro index=False
#No sé si cuando se grabó a disco algo cambió, X-(

df_hits = df_hits.drop([df_hits.columns[0]], axis='columns')

In [8]:
df_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63520301 entries, 0 to 63520300
Data columns (total 16 columns):
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
#hit               int64
lin_det            int64
piso_det           int64
mód_opt            int64
X'                 float64
Y'                 float64
Z'                 float64
azimut             float64
zenit              float64
tiempo             float64
amp                float64
frec_hits          float64
dtypes: float64(8), int64(8)
memory usage: 7.6 GB


In [9]:
df_hits.head(10)

,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,amp,frec_hits
0,25800,73557,0,2,76,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,296.559100,1.089169,327.587128
1,25800,73557,0,2,77,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,137.146191,0.879543,327.587128
2,25800,73557,0,2,78,4,19,1,44.584223,9.539780,90.040108,2.292014,2.356193,153.189412,0.772948,281.085968
3,25800,73557,0,2,67,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,168.470446,1.070132,82.550049
4,25800,73557,0,2,68,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,897.750991,0.985427,82.550049
5,25800,73557,0,2,69,4,20,1,44.485280,9.426792,104.567108,2.552615,2.356193,167.488297,4.492627,73.738098
6,25800,73557,0,2,70,4,20,2,44.928075,8.529054,104.567108,-1.636175,2.356193,171.860943,1.107187,89.874268
7,25800,73557,0,2,71,4,21,0,44.788333,9.655823,119.094108,1.882990,2.356193,201.236802,16.856011,60.100555
8,25800,73557,0,2,72,4,21,1,44.578323,8.677005,119.094108,-2.305800,2.356193,204.882482,3.327063,52.337646
9,25800,73557,0,2,73,4,21,2,45.530937,8.984459,119.094108,-0.211405,2.356193,1035.010812,0.580996,61.416626


In [10]:
#Contrastando con el dataframe obtenido para el punto 3.1 del email de Salva, abrimos el segundo dataframe
df_aafit = pd.read_csv(file_aafit)

In [11]:
#Obtenemos la info del segundo dataframe
df_aafit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2261213 entries, 0 to 2261212
Data columns (total 14 columns):
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
aafit_azimut       float64
aafit_zenit        float64
aafit_lambda       float64
aafit_beta         float64
bbfit_azimut       float64
bbfit_zenit        float64
bbfit_quality      float64
muon_azimut        float64
muon_zenit         float64
muon_energy        float64
dtypes: float64(10), int64(4)
memory usage: 241.5 MB


In [12]:
df_aafit.head(10)

,runID,frameID,trigger_counter,interactionID,aafit_azimut,aafit_zenit,aafit_lambda,aafit_beta,bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy
0,25800,73557,0,2,-2.345721,0.434090,-5.869856,0.130539,-9.424778,0.306723,0.943146,-1.787980,0.339902,15.9965
1,25800,66099,2,2,1.572092,0.533743,-5.779327,0.083667,-9.424778,0.424040,0.843271,1.702232,0.565575,50.3398
2,25800,95343,3,2,0.491548,1.183076,-5.751991,0.020210,-9.424778,0.318698,0.494769,-3.026856,0.314316,22.1763
3,25800,160419,4,2,1.300769,0.186540,-6.103379,0.007983,-9.424778,0.140361,0.881044,1.447459,0.296681,44.5736
4,25800,8919,5,2,2.791342,0.743009,-5.109426,0.009720,2.794801,0.727146,0.933982,2.819365,0.737831,49.0134
5,25800,81967,6,2,1.456237,0.794121,-5.545071,0.020967,1.582917,0.775308,1.369833,1.372126,0.725714,43.6112
6,25800,81971,7,2,2.814358,0.731503,-6.365585,0.008185,-9.424778,0.656407,1.143601,-1.587605,0.674739,30.8739
7,25800,3601,8,2,-0.894596,0.579036,-5.307784,0.010676,-9.424778,0.252441,3.122377,-0.151505,0.616614,50.1583
8,25800,123817,10,2,2.260261,0.673821,-5.522124,0.011747,2.225389,0.664657,1.053378,1.899561,0.579428,34.3649
9,25800,130509,11,2,-0.782032,1.054763,-5.407636,0.041568,-9.424778,0.391115,1.040440,2.218007,0.568050,34.9187


In [13]:
temp = df_aafit['runID'].unique()
print(len(temp))
temp
#En esta ultima corrida, ya aparecen desde el 25800 hasta el 85730, lo cual significa que recorrio todos los archivos.

1375


array([25800, 25880, 25920, ..., 85690, 85720, 85730], dtype=int64)

In [15]:
#Mostramos las 10 primeras líneas del segundo dataframe
#La idea es contrastar que tenemos respecto al runID 25800 en ambos dataframes
print(df_aafit[df_aafit['runID']==25800].shape)

df_hits[df_hits['runID']==25800].shape

(479, 14)


(18487, 16)